# Past Election Results

Process historical election result CSVs from DCBOE from 2012 to 2018

In [1]:
import os
os.chdir('../')
import pandas as pd
from datetime import datetime
from scripts.common import hash_dataframe, match_names

In [2]:
file_root = 'data/dcboe/election_results/'

year_files = {
    2012: 'November_6_2012_General_and_Special_Election_Certified_Results.csv'
    , 2014: 'November_4_2014_General_Election_Certified_Results.csv'
    , 2016: 'November_8_2016_General_Election_Certified_Results.csv'
    , 2018: 'November_6_2018_General_Election_Certified_Results.csv'
}

In [3]:
all_contests = {}
anc_by_year = {}

for year in year_files:
    all_contests[year] = pd.read_csv(file_root + year_files[year])
    all_contests[year] = all_contests[year].rename(columns={
        'Candidate': 'candidate_name'
        , 'CANDIDATE': 'candidate_name'
        , 'ContestName': 'contest_name'
        , 'CONTEST_NAME': 'contest_name'
        , 'ContestNumber': 'contest_number'
        , 'CONTEST_ID': 'contest_number'
        , 'Votes': 'votes'
        , 'VOTES': 'votes'
    })
    
    # Exclude the Over Vote and Under Vote rows
    all_contests[year] = all_contests[year][
        (~all_contests[year]['candidate_name'].str.contains('UNDER VOTES').fillna(False))
        & (~all_contests[year]['candidate_name'].str.contains('OVER VOTES').fillna(False))
    ].copy()

    # Include only ANC contests
    anc_by_year[year] = all_contests[year][
        (all_contests[year]['contest_name'].str.contains('SINGLE MEMBER DISTRICT'))
        | (all_contests[year]['contest_name'].str.contains('ADVISORY NEIGHBORHOOD COMMISSIONER'))
    ].copy()



### Top race in each year

In [4]:
for year in year_files:
    votes_by_contest = all_contests[year].groupby('contest_name').votes.sum().sort_values(ascending=False)
    print(f'\n{year}')
    print(votes_by_contest.head())


2012
contest_name
REGISTERED VOTERS - TOTAL                                483775
AT - LARGE MEMBER OF THE COUNCIL DISTRICT OF COLUMBIA    387131
BALLOTS CAST - TOTAL                                     294814
PRESIDENT AND VICE PRESIDENT DISTRICT OF COLUMBIA        293764
DELEGATE U.S. HOUSE OF REPRESENTATIVES DISTRICT OF       278563
Name: votes, dtype: int64

2014
contest_name
REGISTERED VOTERS - TOTAL                                461325
AT - LARGE MEMBER OF THE COUNCIL DISTRICT OF COLUMBIA    272869
BALLOTS CAST - TOTAL                                     177377
MAYOR OF THE DISTRICT OF COLUMBIA                        175270
DELEGATE U.S. HOUSE OF REPRESENTATIVES                   171893
Name: votes, dtype: int64

2016
contest_name
REGISTERED VOTERS - TOTAL                                   478688
AT - LARGE MEMBER OF THE COUNCIL DISTRICT OF COLUMBIA       443144
BALLOTS CAST - TOTAL                                        312575
ELECTORS OF PRESIDENT AND VICE PRESIDENT          

In [5]:
for year in year_files:
    if year <= 2014:
        anc_by_year[year]['smd_id'] = 'smd_' + anc_by_year[year]['contest_name'].str[35:40]
    else:
        anc_by_year[year]['smd_id'] = 'smd_' + anc_by_year[year]['contest_name'].str[6:10]

In [6]:
anc_by_year[2012].groupby('candidate_name').votes.sum().sort_values(ascending=False).head(10)

candidate_name
WRITE-IN                    8406
PHILLIP J. HAMMOND          1172
DWAYNE M. TOLIVER           1113
BRIAN FLAHAVEN              1072
ROBERT T. (BOB) RICHARDS    1055
JO-ANNE PRUE                1054
ADRIAN JORDAN               1028
ROBERT A. JORDAN            1027
GALE BLACK                  1018
YVONNE A. JEFFERSON          973
Name: votes, dtype: int64

In [7]:
anc = pd.concat(anc_by_year, names=['year']).reset_index()
anc['candidate_name'] = anc['candidate_name'].str.title()

In [8]:
anc.groupby('year').votes.sum().map('{:,d}'.format)

year
2012    206,425
2014    131,666
2016    244,608
2018    190,628
Name: votes, dtype: object

## Name Cleanup

In [9]:
anc['candidate_name'] = anc['candidate_name'].str.strip()

In [10]:
# Before name cleanup
anc.candidate_name.nunique()

1069

In [11]:
## Standardize some names

names_to_standardize = {
    'Write-In': 'Write-in'
    , 'Ernest Johnson': 'Ernest E. Johnson'
    , 'Kent Boese': 'Kent C. Boese'
    , 'Rashida E. Brown': 'Rashida Brown'
    , 'Jon Squicciarini': 'Jon D. Squicciarini'
    , 'Dyana Nm Forester': 'Dyana Forester'
    , 'Jessica Laura Smith': 'Jessica Smith'
    , 'Jessica L. Smith': 'Jessica Smith'
    , 'John C. Green': 'John Carlos Green'
    , "'Franko' Frank O. Agbro": 'Frank O. Agbro'
    , 'Franko Frank O. Agbro': 'Frank O. Agbro'
    , 'William K Smith': 'William Kennedy Smith'
    , 'William K. Smith': 'William Kennedy Smith'
    , 'Florence Harmon': 'Florence E. Harmon'
    , 'Detrick D Campbell': 'Detrick Campbell'
    , 'Randy D Downs': 'Randy D. Downs'
    , 'Jeffrey Jones': 'Jeffrey L. Jones'
    , 'Monica L. Roache': 'Monica L. Roaché'
    , 'Brian Turmail': 'Brian G. Turmail'
    , "Thomas \"Tom\" Smith": 'Tom Smith'
    , 'Jameson Freeman': 'Jameson Paul Freeman'
    , 'Mike N. Gold': 'Michael Gold'
    , 'Karen Lee Perry': 'Karen Perry'
    , 'Chanda Tuck Garfield': 'Chanda Tuck-Garfield'
    , 'Chanda Tuck - Garfield': 'Chanda Tuck-Garfield'
    , 'Jim Mccarthy': 'Jim McCarthy'
    , 'Phyllis C. Green': 'Phyllis Caudle Green'
    , 'Phyllis Green': 'Phyllis Caudle Green'
    , 'Stephen A "Steve" Whatley': 'Stephen A. "Steve" Whatley'
    , 'Dave Wilson': 'David Wilson'
    , 'Brenda Parks': 'Brenda Dawson Parks'
    , 'Ron Austin': 'Ronald (Ron) Austin'
    , 'Tischa M. Cockrell': 'Tischa Cockrell'
    , 'Vann-Di M. Galloway': 'Vann-Di Galloway'
    , 'Renee L. Bowser': 'Renée L. Bowser'
    , 'Renee  L. Bowser': 'Renée L. Bowser'
    , 'Adrian Jordan': 'Adrian T. Jordan'
    , 'Sandra "Sandi" Washington': 'Sandi Washington'
    , 'Gordon - Andrew Fletcher': 'Gordon-Andrew Fletcher'
    , 'Gayle E. Hall-Carley': 'Gayle Carley'
    , 'Gayle Hall Carley': 'Gayle Carley'
    , 'Gayle Hall-Carley': 'Gayle Carley'
    , 'Gail A. Brevard': 'Gail Brevard'
    , 'Robert Looper Iii': 'Robert Looper III'
    , 'Robert Looper, Iii': 'Robert Looper III'
    , 'Peta - Gay S. Lewis': 'Peta-Gay Lewis'
    , 'Peta-Gay S. Lewis': 'Peta-Gay Lewis'
    , 'Bernice S. Blacknell': 'Bernice S Blacknell'
    , 'Deborah "Debbie Smith" Steiner': 'Debbie Smith-Steiner'
    , 'William B. Boston': 'William Boston'
    , 'Christy \'Love\' Davis': 'Christy "Love" Davis'
    , 'Mary Lois Farmer-Allen': 'Mary L. Farmer-Allen'
    , 'Joyce Robinson - Paul': 'Joyce Robinson-Paul'
    , 'Katherine Mcclelland': 'Katherine McClelland'
    , 'Teri Janine Tj Quinn': 'Teri Janine "Tj" Quinn'
    , 'Bertha Holliday': 'Bertha G. Holliday'
    , 'Betsy Mcdaniel': 'Betsy McDaniel'
    , 'C Dianne Barnes': 'Dianne Barnes'
    , 'C. Dianne Barnes': 'Dianne Barnes'
    , 'Nancy "Darlene" Jones': 'Nancy Darlene Jones'
    , 'Nancy D. Jones': 'Nancy Darlene Jones'
    , 'Michael B Soderman': 'Michael Soderman'
    , 'Mike Soderman': 'Michael Soderman'
    , 'Patrick M. Malone': 'Patrick Malone'
    , 'H. J. Amons Sr': 'H.J. Amons Sr.'
    , 'H.J. Amons Sr': 'H.J. Amons Sr.'
    , 'Jennifer E Samolyk': 'Jennifer E. Samolyk'
    , 'Gerald (Jerry) Sroufe': 'Gerald Sroufe'
    , 'Gerald "Jerry" Sroufe': 'Gerald Sroufe'
    , 'James M Loots': 'James M. Loots'
    , 'Karen J. Wirt': 'Karen Wirt'
    , 'John Scott Price': 'Scott Price'
    , 'Tony T Goodman': 'Tony T. Goodman'
    , 'Ron Mcbee': 'Ron McBee'
    , 'Rhonda Natalie Hamilton': 'Rhonda N. Hamilton'
    , 'Kevin L Chapple': 'Kevin Chapple'
    , 'Leroy Thorpe': 'Leroy Joseph Thorpe Jr.'
    , 'Leroy Joseph Thorpe, Jr.': 'Leroy Joseph Thorpe Jr.'
    , 'Frank Wiggins': 'Frank S. Wiggins'
    , 'Alvin Judd Sr': 'Alvin Judd Sr.'
    , 'Alvin Judd, Sr.': 'Alvin Judd Sr.'
    , 'D. Lynnell Humphrey': 'D.L. Humphrey'
    , 'Patricia Malloy': 'Patricia A. Malloy'
    , 'Veronica Raglin': 'Veronica E. Raglin'
    , 'George B. Browne, Jr.': 'George B. Browne Jr.'
    , 'Jean M. Mcvea': 'Jean M. McVea'
    , 'Linda Holmes': 'Linda M. Holmes'
    , 'Evie L. Washington': 'Evie Washington'
    , 'Sheila M Carson Carr': 'Sheila Carson Carr'
    , 'Sheila M. Carson Carr': 'Sheila Carson Carr'
    , 'Sheila Marie Carson Carr': 'Sheila Carson Carr'
    , 'Troy Donté Prestwood': 'Troy Donte Prestwood'
    , 'Anthony Lorenzo': 'Anthony Lorenzo Green'
    , 'Jeffrey Mccoy': 'Jeffrey McCoy'
    , 'Grady Edwards Iii': 'Grady Edwards III'
    , 'Rhonda Edwards-Hines': 'Rhonda L. Edwards-Hines'
    , 'Rhonda L. Edwards - Hines': 'Rhonda L. Edwards-Hines'
    , 'Patricia (Pat) Carmon': 'Patricia "Pat" Carmon'
    , 'Patricia Pat Carmon': 'Patricia "Pat" Carmon'
    , 'Joseph Johnson': 'Joseph B. Johnson'
    , 'Kendall L. Simmons': 'Kendall Simmons'
    , 'Sandra (Sv) Williams': 'S V Williams'
    , 'Stephen A Slaughter': 'Stephen A. Slaughter'
    , 'Deborah R. Thomas': 'Deborah Thomas'
    , 'Karlene (K.) Armstead': 'Karlene "K" Armstead'
    , 'Constance "Connie" Mobley': 'Constance Mobley'
    , 'Joshua R. Mater': 'Joshua Ryan Mater'
}

anc['candidate_name'] = anc['candidate_name'].replace(names_to_standardize)
len(names_to_standardize)

107

In [12]:
# After name cleanup
anc.candidate_name.nunique()

982

## Group by Candidates

Results are split out over precincts. Group by candidates to know who won each race. 

In [13]:
candidates = anc.groupby(['year', 'smd_id', 'contest_number', 'candidate_name']).votes.sum().reset_index()

In [14]:
candidates['ranking'] = candidates.groupby(['year', 'smd_id']).votes.rank(method='first', ascending=False)
candidates['winner'] = candidates['ranking'] == 1

In [15]:
candidates['write_in_winner'] = candidates['winner'] & (candidates['candidate_name'] == 'Write-in')

In [16]:
candidates = candidates.sort_values(by=['year', 'smd_id', 'votes'], ascending=[True, True, False])

In [17]:
total_votes = candidates.groupby(['year', 'smd_id']).votes.sum()
total_votes.name = 'total_votes'
candidates = pd.merge(candidates, total_votes, how='inner', on=['year', 'smd_id'])
candidates['vote_share'] = candidates['votes'] / candidates['total_votes']

In [18]:
candidates #.to_clipboard()

,year,smd_id,contest_number,candidate_name,votes,ranking,winner,write_in_winner,total_votes,vote_share
0,2012,smd_1A01,117,Lisa Kralovic,374,1.0,True,False,398,0.939698
1,2012,smd_1A01,117,Write-in,24,2.0,False,False,398,0.060302
2,2012,smd_1A02,118,Vickey A. Wright-Smith,432,1.0,True,False,738,0.585366
3,2012,smd_1A02,118,Alexander Gallo,295,2.0,False,False,738,0.399729
4,2012,smd_1A02,118,Write-in,11,3.0,False,False,738,0.014905
...,...,...,...,...,...,...,...,...,...,...
2792,2018,smd_8E05,609,Write-in,41,2.0,False,False,566,0.072438
2793,2018,smd_8E06,610,"Karlene ""K"" Armstead",378,1.0,True,False,395,0.956962
2794,2018,smd_8E06,610,Write-in,17,2.0,False,False,395,0.043038
2795,2018,smd_8E07,611,Stephen A. Slaughter,455,1.0,True,False,495,0.919192


## Results in one SMD

In [19]:
candidates[candidates['smd_id'] == 'smd_1B08'][['year', 'candidate_name', 'votes', 'ranking']]

,year,candidate_name,votes,ranking
45,2012,Emily Washington,640,1.0
46,2012,Write-in,34,2.0
727,2014,Mark Ranslem,350,1.0
728,2014,Write-in,24,2.0
1432,2016,Joshua Ryan Mater,785,1.0
1433,2016,Write-in,30,2.0
2145,2018,Joshua Ryan Mater,581,1.0
2146,2018,Write-in,14,2.0


In [20]:
candidates[candidates['candidate_name'].str.contains('Thomas')][['year', 'candidate_name', 'votes', 'ranking']]

,year,candidate_name,votes,ranking
16,2012,Thomas Boisvert,633,1.0
37,2012,Deborah Thomas,713,1.0
211,2012,W. Philip Thomas,355,1.0
264,2012,Sandra Thomas,214,3.0
354,2012,Timothy Thomas,356,3.0
372,2012,Thomas E. Redmond,265,2.0
383,2012,Marjorie D. Thomas-Barnes,434,1.0
420,2012,Bradley Ashton Thomas,405,2.0
773,2014,Thomas B. Martin,196,2.0
1046,2014,Timothy Thomas,333,2.0


## Save historical candidate names for matching

In [21]:
candidates['dcboe_hash_id'] = hash_dataframe(candidates, ['smd_id', 'candidate_name'])

In [22]:
candidates_to_match = candidates.loc[
    ~candidates['candidate_name'].isin(['Write-in', 'No Candidate Filed'])
    , ['dcboe_hash_id', 'smd_id', 'candidate_name', 'year']
].groupby(['dcboe_hash_id', 'smd_id', 'candidate_name']).agg(
    num_candidacies=('year', 'size')
    , first_year=('year', 'min')
).reset_index()
# candidates_to_match_series.name = 'num_candidacies'
# candidates_to_match = candidates_to_match_series.reset_index()
candidates_to_match

,dcboe_hash_id,smd_id,candidate_name,num_candidacies,first_year
0,009709d190e37c9f0feaa25da16fad4aff6d4922bdb772...,smd_1B01,Jonathan Goldman,1,2016
1,016b69abd9c5c059b323beaf3ba2127a2262b4512c76bf...,smd_5A03,Lionell K. Gaines,1,2016
2,017bb584db84d24e84b54e35e5574df7b64d1515f71362...,smd_3D04,Stu Ross,2,2012
3,01c272c6456c5c7139e6a41f6df673e382700e3cb1bb3b...,smd_5E07,Bertha G. Holliday,3,2014
4,023522ab0c2888fc030beb2f5473cd6e59ea62f615ea52...,smd_7D05,Jamila K. Yore,1,2012
...,...,...,...,...,...
983,fe5c80bea67c90da1effb1798526a425f28d3d75f02622...,smd_5E03,Michael Aiello,1,2016
984,fef1af28a47e4d6a2caa74b746660eef4b499fe43609a7...,smd_6C04,Mark Eckenwiler,4,2012
985,ff54da7b1619800a983c29889a80ddba0d8f9808036891...,smd_6A06,Peter Grant,1,2016
986,ff70bfd23abb89931d9ee0865ad4d75fed87a5fc7ffdf4...,smd_6C06,Shree Chauhan,1,2012


In [23]:
# candidates_to_match.to_csv('data/candidates_to_match.csv', index=False)

## Match to People

Originally, I did this in a separate workbook called Match Candidate Names.

In [24]:
people = pd.read_csv('data/people.csv')

In [25]:
# Create a column for integers, starting will NULLs, where the person_id from the matches will go
candidates_to_match['person_id'] = pd.NA
candidates_to_match['person_id'] = candidates_to_match['person_id'].astype('Int64')

people['person_id'] = people['person_id'].astype('Int64')

In [26]:
for idx, row in candidates_to_match.iterrows():
    
    people_smd = people[people['any_smd'] == row['smd_id']].copy()
    
    best_id, best_score = match_names(row['candidate_name'], people_smd['full_name'], people_smd['person_id'])
    
    if best_score >= 80:
        candidates_to_match.loc[idx, 'match_score'] = best_score
        candidates_to_match.loc[idx, 'person_id'] = best_id
        candidates_to_match.loc[idx, 'match_name'] = people.loc[people.person_id == best_id, 'full_name'].iloc[0]
        candidates_to_match.loc[idx, 'match_smd_id'] = people.loc[people.person_id == best_id, 'any_smd'].iloc[0]

In [27]:
# This person has run and won in 2 different SMDs
candidates_to_match.loc[candidates_to_match['candidate_name'] == 'Anthony Lorenzo Green', 'person_id'] = 10227
candidates_to_match.loc[candidates_to_match['candidate_name'] == 'Anthony Lorenzo Green']

,dcboe_hash_id,smd_id,candidate_name,num_candidacies,first_year,person_id,match_score,match_name,match_smd_id
14,066f6bc639a2296351f29d230c60506a661485beef2547...,smd_8B04,Anthony Lorenzo Green,2,2012,10227,NaN,NaN,NaN
565,94aa6bf43202356ced3ffe79b90da22607bd8a1d821ed4...,smd_7C04,Anthony Lorenzo Green,2,2016,10227,100.0,Anthony Lorenzo Green,smd_7C04


In [28]:
print('Matches found: {}'.format(candidates_to_match.person_id.nunique()))

Matches found: 986


## Insert into People table

In [29]:
# candidates_to_insert = candidates_to_match[candidates_to_match.person_id.isnull()].sort_values(
#     by=['smd_id', 'first_year', 'candidate_name'])


# existing_max_id = 10635 # people.person_id.max()
# candidates_to_insert['person_id'] = range(existing_max_id+1, existing_max_id+1+len(candidates_to_insert))

# # Copy this to the `people` tab in Source Google Sheet
# candidates_to_insert[['person_id', 'candidate_name', 'smd_id']].to_clipboard(index=False)

## Insert into Commissioners table

I don't think I'll ever know who the write-in winners from past elections actually were. 

In [30]:
# Dataframe that contains historical candidate records AND person_id
candidates_people = pd.merge(
    candidates
    , candidates_to_match[['dcboe_hash_id', 'person_id', 'match_name']]
    , how='left'
    , on='dcboe_hash_id'
)
candidates_people = candidates_people.rename(columns={'match_name': 'full_name'})

In [31]:
people = pd.read_csv('data/people.csv')
commissioners = pd.read_csv('data/commissioners.csv')
commissioners['start_date'] = pd.to_datetime(commissioners['start_date'])
commissioners['end_date'] = pd.to_datetime(commissioners['end_date'])

In [32]:
winners = candidates_people[
    (candidates_people.winner)
    & ~(candidates_people.write_in_winner)
#     & (candidates_people.year < 2018)
].copy()

In [33]:
cycle_start_date = {
    2012: datetime(2013,1,2)
    , 2014: datetime(2015,1,2)
    , 2016: datetime(2017,1,2)
    , 2018: datetime(2019,1,2)
}

cycle_end_date = {
    2012: datetime(2015,1,2)
    , 2014: datetime(2017,1,2)
    , 2016: datetime(2019,1,2)
    , 2018: datetime(2021,1,2)
}
winners['start_date'] = winners['year'].map(cycle_start_date)
winners['end_date'] = winners['year'].map(cycle_end_date)

In [34]:
# Concat past winners with existing commissioner list
winners['is_chair'] = pd.NA
winners['notes'] = pd.NA
winners['commissioner_name'] = winners['candidate_name']
winners['source'] = 'winners'
commissioners['source'] = 'commissioners'

columns_to_concat = [
    'person_id'
    , 'smd_id'
    , 'commissioner_name'
    , 'start_date'
    , 'end_date'
    , 'is_chair'
    , 'notes'
    , 'source'
]

comm_winners = pd.concat([commissioners[columns_to_concat], winners[columns_to_concat]])
comm_winners

,person_id,smd_id,commissioner_name,start_date,end_date,is_chair,notes,source
0,10001,smd_1A01,Layla Bonnot,2019-01-02,2023-01-02,NaN,NaN,commissioners
1,10002,smd_1A02,Teresa A. Edmondson,2019-01-02,2021-01-02,NaN,NaN,commissioners
2,10003,smd_1A03,Zach Rybarczyk,2019-01-02,2021-08-31,NaN,asked for actual date,commissioners
3,10004,smd_1A04,Matthew Goldschmidt,2019-01-02,2021-01-02,NaN,NaN,commissioners
4,10005,smd_1A05,Christine Miller,2019-01-02,2023-01-02,NaN,NaN,commissioners
...,...,...,...,...,...,...,...,...
2786,10280,smd_8E03,Monica Watts,2019-01-02,2021-01-02,<NA>,<NA>,winners
2788,10281,smd_8E04,Kendall Simmons,2019-01-02,2021-01-02,<NA>,<NA>,winners
2791,10282,smd_8E05,Christopher L. Hawthorne,2019-01-02,2021-01-02,<NA>,<NA>,winners
2793,10283,smd_8E06,"Karlene ""K"" Armstead",2019-01-02,2021-01-02,<NA>,<NA>,winners


In [35]:
comm_winners = comm_winners.sort_values(by=['smd_id', 'person_id', 'start_date'])

In [36]:
comm_winners['previous_start_date'] = comm_winners.groupby(['smd_id', 'person_id']).start_date.shift(1)
comm_winners['previous_end_date'] = comm_winners.groupby(['smd_id', 'person_id']).end_date.shift(1)

In [37]:
comm_winners['days_from_previous_end'] = (comm_winners['start_date'] - comm_winners['previous_end_date']).dt.days

In [38]:
comm_winners[['person_id'
    , 'smd_id'
    , 'commissioner_name'
    , 'start_date'
    , 'end_date'
    , 'previous_start_date'
    , 'previous_end_date'
    , 'days_from_previous_end'
    , 'source'
]].sort_values(by=['smd_id', 'start_date', 'person_id', 'source']) #.to_clipboard(index=False)

In [39]:
comm_winners[comm_winners.commissioner_name == 'Deborah Thomas']

,person_id,smd_id,commissioner_name,start_date,end_date,is_chair,notes,source,previous_start_date,previous_end_date,days_from_previous_end
37,10016,smd_1B04,Deborah Thomas,2013-01-02,2015-01-02,<NA>,<NA>,winners,NaT,NaT,NaN
15,10016,smd_1B04,Deborah Thomas,2020-01-02,2023-01-02,NaN,She didn't run in the 2018 election but seems ...,commissioners,2013-01-02,2015-01-02,1826.0


In [40]:
candidates[candidates.candidate_name == 'Deborah Thomas']

,year,smd_id,contest_number,candidate_name,votes,ranking,winner,write_in_winner,total_votes,vote_share,dcboe_hash_id
37,2012,smd_1B04,132,Deborah Thomas,713,1.0,True,False,763,0.934469,4e41512d3d293ecc2030fcd93f5276636ea53db44380f2...


In [41]:
comm_winners.person_id.nunique()

834